In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['schema_name'] = "nnfabrik_table_test"
schema = dj.schema("nnfabrik_table_test")

import os
from os import listdir
from os.path import isfile, join

import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle 
import time

In [35]:
import nnfabrik
from nnfabrik import main, builder

import nnvision
from nnvision.tables.main import MonkeyExperiment

In [ ]:
@schema
class TrainedModel(TrainedModelBase):
    table_comment = "Trained models"

In [40]:
MonkeyExperiment.populate()

In [41]:
MonkeyExperiment()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,brain_area some string,experiment_name another string,n_sessions,total_n_neurons,avg_oracle,avg_explainable_var
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,V1,CSRF,5,68,0.490675,0.294426


# Dataset loading Speed comparison

In [55]:
dataset_config = dict(crop=50,
                   subsample=1,
                   seed=1000,
                   batch_size=128)

from nnfabrik.datasets import monkey_static_loader, csrf_v1

## Old dataloader

In [56]:
start = time.time()

imagepath = os.getcwd()+'/data/images/Kelli Restivo - imageSet1_35ppd.pickle'
mypath = os.getcwd()+'/data/'
all_files = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
dataloaders = csrf_v1(datafiles=all_files,
                       imagepath=imagepath,
                       **dataset_config
                       )
print("Time to load old Dataloader: {}s".format(str(time.time()-start)))

Time to load old Dataloader: 133.0452094078064s


In [57]:
start = time.time()
image_cache_path = os.getcwd()+'/data/images/individual/'
image_file = os.getcwd()+'/data/images/Kelli Restivo - imageSet1_35ppd.pickle'
mypath = os.getcwd()+'/data/'
all_files = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]
dataloaders = monkey_static_loader(dataset='CSRF_V1',
                                   neuronal_data_files=all_files,
                                   image_file=image_file,
                                   image_cache_path=image_cache_path,
                                   **dataset_config
                                   )
print("Time to load new Dataloader: {}s".format(str(time.time()-start)))

Time to load new Dataloader: 8.18419885635376s


In [58]:
dataset_fn = 'nnvision.datasets.monkey_static_loader'
dataset_config = dict(dataset='CSRF_V1',
                               neuronal_data_files=all_files[:5],
                               image_file=image_file,
                               image_cache_path=image_cache_path,
                               crop=80,
                               subsample=1,
                               seed=1000,
                               batch_size=128,)
Dataset().add_entry(dataset_fn,dataset_config, dataset_comment="CSRF_V1")

{'dataset_fn': 'nnvision.datasets.monkey_static_loader',
 'dataset_hash': '0f6612894b9ff8e5c02e521dc5269d66',
 'dataset_config': {'dataset': 'CSRF_V1',
  'neuronal_data_files': ['/notebooks/notebooks/data/Kelli Restivo - V1_latency40_session_21_3632669014376.pickle',
   '/notebooks/notebooks/data/Kelli Restivo - V1_latency40_session_22_3632932714885.pickle',
   '/notebooks/notebooks/data/Kelli Restivo - V1_latency40_session_23_3633364677437.pickle',
   '/notebooks/notebooks/data/Kelli Restivo - V1_latency40_session_24_3634055946316.pickle',
   '/notebooks/notebooks/data/Kelli Restivo - V1_latency40_session_26_3634658447291.pickle'],
  'imagepath': '/notebooks/notebooks/data/images/Kelli Restivo - imageSet1_35ppd.pickle',
  'cached_images_path': '/notebooks/notebooks/data/images/individual/',
  'crop': 80,
  'subsample': 1,
  'seed': 1000,
  'batch_size': 128},
 'dataset_fabrikant': 'kwilleke',
 'dataset_comment': 'CSRF_V1'}

In [ ]:
# as of right now, the function monkey static loader is also present in nnfabrik.
# so you can import it as:
from nnfabrik.datasets.sysident_v1_dataset import monkey_static_loader.
# and then add it to the table like this. It's doing the same thing.

dataset_fn = 'nnfabrik.datasets.sysident_v1_dataset.monkey_static_loader'
dataset_config = dict(dataset='CSRF_V1',
                               neuronal_data_files=all_files[:5],
                               image_file=image_file,
                               image_cache_path=image_cache_path,
                               crop=80,
                               subsample=1,
                               seed=1000,
                               batch_size=128,)
Dataset().add_entry(dataset_fn,dataset_config, dataset_comment="CSRF_V1")

# MonkeyExperiment table of nnvision.main

In [2]:
import nnvision
from nnvision.tables.main import MonkeyExperiment

Schema name: nnfabrik_table_test


In [50]:
MonkeyExperiment().Sessions()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,session_id,animal_id,n_neurons
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,31,21
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632932714885,31,11
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3633364677437,31,10
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3634055946316,31,21
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3634658447291,31,5


In [59]:
MonkeyExperiment().UnitStatistics()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,session_id,unit_id,unit_avg_firing,unit_fano_factor
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,289,0.943163,1.53654
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,290,0.181131,2.58351
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,291,2.47268,2.1469
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,292,1.55822,1.75168
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,293,0.172424,2.61695
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,294,0.970338,1.1123
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,295,2.71649,1.1626
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,296,3.06717,1.19091
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,297,1.67381,2.29148
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,298,8.2573,1.84225


In [60]:
MonkeyExperiment.UnitMeasures()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,session_id,unit_id,unit_oracle,unit_explainable_var
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,289,0.359314,0.148836
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,290,0.575999,0.343884
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,291,0.391127,0.170688
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,292,0.557024,0.32357
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,293,0.26224,0.0826758
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,294,0.370259,0.155252
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,295,0.217519,0.0625701
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,296,0.258806,0.0839725
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,297,0.24794,0.0777069
monkey_static_loader,219afafdd04bfa7b47054c46ea939963,3632669014376,298,0.680313,0.474144
